In [23]:

import math # Mathematical functions 
import numpy as np # Fundamental package for scientific computing with Python
import pandas as pd # Additional functions for analysing and manipulating data
from datetime import date, timedelta, datetime # Date Functions
from pandas.plotting import register_matplotlib_converters # This function adds plotting functions for calender dates
import matplotlib.pyplot as plt # Important package for visualization - we use this to plot the market data
import matplotlib.dates as mdates # Formatting dates
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score # Packages for measuring model performance / errors
from tensorflow.keras import Sequential # Deep learning library, used for neural networks
from tensorflow.keras.layers import SimpleRNN, GRU,LSTM, Dense, Dropout,Bidirectional # Deep learning classes for recurrent and regular densely-connected layers
from tensorflow.keras.callbacks import EarlyStopping # EarlyStopping during model training
from sklearn.preprocessing import RobustScaler, MinMaxScaler # This Scaler removes the median and scales the data according to the quantile range to normalize the price data 
import seaborn as sns # Visualization
sns.set_style('white', { 'axes.spines.right': False, 'axes.spines.top': False})
!pip install yfinance
import yfinance as yf #Alternative package if webreader does not work: pip install yfinance


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
def lstm(x_train):
  model = Sequential()

  n_neurons = x_train.shape[1] * x_train.shape[2]
  print(n_neurons, x_train.shape[1], x_train.shape[2])
  # model.add(LSTM(n_neurons, return_sequences=False, input_shape=(x_train.shape[1], x_train.shape[2]))) 
  model.add((GRU(n_neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2]))) )
  model.add((GRU(n_neurons, return_sequences=False)))
  model.add(Dense(5))
  model.add(Dense(1))

  # Compile the model
  model.compile(optimizer='adam', loss='mse')
  return model

In [25]:
def data_collect(): 
  df2=pd.read_csv('/content/Data 5 years - Sheet1.csv')
  df2.set_index('Date', inplace=True, drop=True)
  # df2.drop('Date')
  updated_df = df2.dropna(axis=0)
  # print(updated_df.info())
  # updated_df[250:]
  return updated_df



In [26]:
def data_creation(dataset):
  # Indexing Batches
  df=dataset
  train_df = df.sort_values(by=['Date']).copy()
  FEATURES=df.columns.tolist()
  print('FEATURE LIST')
  print([f for f in FEATURES])

  # Create the dataset with features and filter the data to the list of FEATURES
  data = pd.DataFrame(train_df)
  data_filtered = data[FEATURES]

  # We add a prediction column and set dummy values to prepare the data for scaling
  data_filtered_ext = data_filtered.copy()
  data_filtered_ext['Prediction'] = data_filtered_ext['Close']

  
  return data_filtered_ext,data_filtered,FEATURES,data



In [27]:
def partition_dataset(sequence_length, data,index_close):
    x, y = [], []
    data_len = data.shape[0]
    for i in range(sequence_length, data_len):
        x.append(data[i-sequence_length:i,:]) #contains sequence_length values 0-sequence_length * columsn
        y.append(data[i, index_close]) #contains the prediction values for validation,  for single-step prediction
    
    # Convert the x and y to numpy arrays
    x = np.array(x)
    y = np.array(y)
    return x, y

In [28]:
def data_scaling(data_filtered_ext,data_filtered,Features,data,timesteps):
  # Print the tail of the dataframe
  # data_filtered_ext.tail()
  # Get the number of rows in the data
  nrows = data_filtered.shape[0]

  # Convert the data to numpy values
  np_data_unscaled = np.array(data_filtered)
  np_data = np.reshape(np_data_unscaled, (nrows, -1))
  print(np_data.shape)

  # Transform the data by scaling each feature to a range between 0 and 1
  scaler = MinMaxScaler()
  np_data_scaled = scaler.fit_transform(np_data_unscaled)
  for i in range(len(Features)):
    print(i,Features[i])
  in_str='Close'#input('Enter prediction attribute: ')
  attribute=in_str.capitalize()


  # Creating a separate scaler that works on a single column for scaling predictions
  scaler_pred = MinMaxScaler()
  df_Close = pd.DataFrame(data_filtered_ext[attribute])
  np_Close_scaled = scaler_pred.fit_transform(df_Close)
  # Set the sequence length - this is the timeframe used to make a single prediction
  # sequence_length = 50
  sequence_length = timesteps

  # Prediction Index
  index_close = data.columns.get_loc(attribute)

  # Split the training data into train and train data sets
  # As a first step, we get the number of rows to train the model on 80% of the data 
  train_data_len = math.ceil(np_data_scaled.shape[0] * 0.8)

  # Create the training and test data
  train_data = np_data_scaled[0:train_data_len, :]
  test_data = np_data_scaled[train_data_len - sequence_length:, :]
  # Generate training data and test data
  x_train, y_train = partition_dataset(sequence_length, train_data,index_close)
  x_test, y_test = partition_dataset(sequence_length, test_data,index_close)

  # Print the shapes: the result is: (rows, training_sequence, features) (prediction value, )
  print(x_train.shape, y_train.shape)
  print(x_test.shape, y_test.shape)

  # Validate that the prediction value and the input match up
  # The last close price of the second input sample should equal the first prediction value
  print(x_train[1][sequence_length-1][index_close])
  print(y_train[0])
  return x_train,x_test,y_train,y_test,attribute,scaler,scaler_pred,train_data_len,sequence_length

In [29]:
def training(model,model_save_name,x_train,y_train,x_test,y_test,epochs,timesteps): 
  batch_size = 16
  early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1)
  history = model.fit(x_train, y_train, 
                      batch_size=batch_size, 
                      epochs=epochs,
                      validation_data=(x_test, y_test)
                    )
  # name=model_save_name+"_epochs"+str(epochs)+"_steps"+str(timesteps)+".h5"
  # model.save(name)
  return history,epochs
                      

In [30]:
def pred_train(model,df,x_test,y_test,data_filtered_ext,attribute,FEATURES,end_date,scaler,scaler_pred,train_data_len,sequence_length):
    
  # Get the predicted values
  y_pred_scaled = model.predict(x_test)

  # Unscale the predicted values
  y_pred = scaler_pred.inverse_transform(y_pred_scaled)
  y_test_unscaled = scaler_pred.inverse_transform(y_test.reshape(-1, 1))

  # Mean Absolute Error (MAE)
  MAE = mean_absolute_error(y_test, y_pred_scaled)
  print(f'Mean Absolute Error (MAE): {np.round(MAE, 2)}')

  # Mean Absolute Percentage Error (MAPE)
  MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
  print(f'Mean Absolute Percentage Error (MAPE): {np.round(MAPE, 2)} %')

  # Median Absolute Percentage Error (MDAPE)
  MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled)) ) * 100
  print(f'Median Absolute Percentage Error (MDAPE): {np.round(MDAPE, 2)} %')

  # Root Mean squared Error (MAE)
  # MSE = mean_squared_error(y_test_unscaled, y_pred)
  MSE = np.square(np.subtract(y_test, y_pred_scaled)).mean()
  rmse = math.sqrt(MSE)  
  print(f'Root Mean Squared Error (RMSE): {rmse}')

  #accuracy -r2 score
  accu=r2_score(y_test, y_pred_scaled)
  print(f'Accuracy (r2 score): {accu}')

  return accu

In [31]:
def stock_training():
  df=data_collect()
  data_filtered_ext,data_filtered,Features,data=data_creation(df)
  timesteps=25
  x_train,x_test,y_train,y_test,attribute,scaler,scaler_pred,train_data_len,sequence_length=data_scaling(data_filtered_ext,data_filtered,Features,data,timesteps)
  
  optimizer='adam'#input('Enter Optimizer: ')
  loss='mae'#input('Enter Loss: ')
  # epochs=int(input('Enter number of epochs: '))
  epochs=10


  model_lstm_1=lstm(x_train)



  # model_hybrid = hybrid(x_train,optimizer,loss)
  print('-------------------------------Training Start LSTM_1---------------------------',end="\n")
  history_lstm_1,epochs_lstm_1=training(model_lstm_1,'lstm_1',x_train,y_train,x_test,y_test,epochs,timesteps)

 
  end_date =  '2022-12-31'

  print('-------------------------------Prediction lstm 1---------------------------',end="\n")
  accu=pred_train(model_lstm_1,df,x_test,y_test,data_filtered_ext,attribute,Features,end_date,scaler,scaler_pred,train_data_len,sequence_length)

  return accu
 

In [32]:
def fitness_function(positions,counter,df_dict):
    features = np.where(positions>=0.4999)[0]
    acc=stock_training()
    accuracy = acc
    temp=[counter,acc]
    df_dict.loc[len(df_dict)]=temp

    counter+=1
    w = 0.9
    leng=23#len(features)
    # if(leng==0): leng=1
    return -(w*accuracy + (1-w) * 1/leng),counter,df_dict

In [33]:
# import random
# import numpy
# import math
# import time
    
# def tanh_jBGWOupdate(X,Bstep):
#   if(X+Bstep)>=1:
#     Y=1
#   else:
#     Y=-1
#   return Y

# def tanh_jBstepBGWO(AD):#  def tanhupdate(AD):
#   x=10 * (AD - 0.5)
#   Cstep = (math.exp(x)-math.exp(-x))/(math.exp(x)+math.exp(-x));
#   if Cstep >= random.random():
#     Bstep = 1 
#   else:
#     Bstep = -1
#   return Bstep


# # def jBstepBGWO(AD):
# #   x=10 * (AD - 0.5)
# #   Cstep = 1 / (1 + math.exp(-x)); 
# #   if Cstep >= random.random():
# #     Bstep = 1 
# #   else:
# #     Bstep = 0
# #   return Bstep

# def tanh_GWO(objf,lb,ub,dim,SearchAgents_no,Max_iter,counter,df_dict):
    
    
#     #Max_iter=1000
#     #lb=-100
#     #ub=100
#     #dim=30  
#     #SearchAgents_no=5
    
#     # initialize alpha, beta, and delta_pos
#     Alpha_pos=numpy.zeros(dim)
#     Alpha_score=float("inf")
    
#     Beta_pos=numpy.zeros(dim)
#     Beta_score=float("inf")
    
#     Delta_pos=numpy.zeros(dim)
#     Delta_score=float("inf")

#     if not isinstance(lb, list):
#         lb = [lb] * dim
#     if not isinstance(ub, list):
#         ub = [ub] * dim
    
#     #Initialize the positions of search agents
#     Positions = numpy.zeros((SearchAgents_no, dim))
#     for i in range(dim):
#         Positions[:, i] = numpy.random.uniform(0,1, SearchAgents_no) * (ub[i] - lb[i]) + lb[i]
    
#     Convergence_curve=numpy.zeros(Max_iter)

#      # Loop counter
#     print("GWO is optimizing  \""+objf.__name__+"\"")    
    
#     timerStart=time.time() 
#     # Main loop
#     for l in range(0,Max_iter):
#         for i in range(0,SearchAgents_no):
            
#             # Return back the search agents that go beyond the boundaries of the search space
#             for j in range(dim):
#                 Positions[i,j]=numpy.clip(Positions[i,j], lb[j], ub[j])

#             # Calculate objective function for each search agent
#             fitness,counter,df_dict=objf(Positions[i,:],counter,df_dict)
            
#             # Update Alpha, Beta, and Delta
#             if fitness<Alpha_score :
#                 Alpha_score=fitness; # Update alpha
#                 Alpha_pos=Positions[i,:].copy()
            
            
#             if (fitness>Alpha_score and fitness<Beta_score ):
#                 Beta_score=fitness  # Update beta
#                 Beta_pos=Positions[i,:].copy()
            
            
#             if (fitness>Alpha_score and fitness>Beta_score and fitness<Delta_score): 
#                 Delta_score=fitness # Update delta
#                 Delta_pos=Positions[i,:].copy()
            
        
        
        
#         a=2-l*((2)/Max_iter); # a decreases linearly from 2 to 0
        
#         # Update the Position of search agents including omegas
#         for i in range(0,SearchAgents_no):
#             for j in range (0,dim):     
#     #--------------------------For Alpha-------------------------------
#                 r1=random.random() # r1 is a random number in [0,1]
#                 r2=random.random() # r2 is a random number in [0,1]
                
#                 A1=2*a*r1-a; # Equation (3.3)
#                 C1=2*r2; # Equation (3.4)
                
#                 D_alpha=abs(C1*Alpha_pos[j]-Positions[i,j]); # Equation (3.5)-part 1
          
#                 Bstep1 = tanh_jBstepBGWO(A1 * D_alpha);#my change
          
#                 # X1=Alpha_pos[j]-A1*D_alpha; # Equation (3.6)-part 1
#                 X1 = tanh_jBGWOupdate(Alpha_pos[j],Bstep1);
      
#     #--------------------------For Beta-------------------------------                       
#                 r1=random.random()
#                 r2=random.random()
                
#                 A2=2*a*r1-a; # Equation (3.3)
#                 C2=2*r2; # Equation (3.4)
                
#                 D_beta=abs(C2*Beta_pos[j]-Positions[i,j]); # Equation (3.5)-part 2
#                 Bstep2 = tanh_jBstepBGWO(A2 * D_beta);#my change 
#                 # X2=Beta_pos[j]-A2*D_beta; # Equation (3.6)-part 2       
#                 X2 = tanh_jBGWOupdate(Beta_pos[j],Bstep2);
    
#     #--------------------------For Delta-------------------------------            
#                 r1=random.random()
#                 r2=random.random() 
                
#                 A3=2*a*r1-a; # Equation (3.3)
#                 C3=2*r2; # Equation (3.4)
                
#                 D_delta=abs(C3*Delta_pos[j]-Positions[i,j]); # Equation (3.5)-part 3
#                 Bstep3 = tanh_jBstepBGWO(A3 * D_delta);#my change
#                 X3=Delta_pos[j]-A3*D_delta; # Equation (3.5)-part 3   
#                 X3 = tanh_jBGWOupdate(Delta_pos[j],Bstep3);

#     #--------------------------Final updation in X-------------------------------
#                 # Positions[i,j]=(X1+X2+X3)/3  # Equation (3.7)
#                 r=random.random()
#                 if r < -1/3:
#                   Positions[i,j] = X1
#                 elif r < 1/3  and  r >=-1/3:
#                   Positions[i,j] = X2
#                 else:
#                   Positions[i,j] = X3
                
            
        
        
#         Convergence_curve[l]=Alpha_score;

#         if (l%1==0):
#                print(['At iteration '+ str(l)+ ' the best fitness is '+ str(Alpha_score)]);
#                print('alpha:', numpy.where(Alpha_pos>0.5)[0])
    
#     timerEnd=time.time()
#     print('Completed in', (timerEnd - timerStart))
    
    
#     return Alpha_pos,counter,df_dict
    

In [34]:
import random
import numpy
import math
import time
    
def sig_jBGWOupdate(X,Bstep):
  if(X+Bstep)>=1:
    Y=1
  else:
    Y=0
  return Y

def sig_jBstepBGWO(AD):
  Cstep = 1 / (1 + math.exp(-10 * (AD - 0.5))); 
  if Cstep >= random.random():
    Bstep = 1 
  else:
    Bstep = 0
  return Bstep

def sig_GWO(objf,lb,ub,dim,SearchAgents_no,Max_iter,counter,df_dict):
    
    
    #Max_iter=1000
    #lb=-100
    #ub=100
    #dim=30  
    #SearchAgents_no=5
    
    # initialize alpha, beta, and delta_pos
    Alpha_pos=numpy.zeros(dim)
    Alpha_score=float("inf")
    
    Beta_pos=numpy.zeros(dim)
    Beta_score=float("inf")
    
    Delta_pos=numpy.zeros(dim)
    Delta_score=float("inf")

    if not isinstance(lb, list):
        lb = [lb] * dim
    if not isinstance(ub, list):
        ub = [ub] * dim
    
    #Initialize the positions of search agents
    Positions = numpy.zeros((SearchAgents_no, dim))
    for i in range(dim):
        Positions[:, i] = numpy.random.uniform(0,1, SearchAgents_no) * (ub[i] - lb[i]) + lb[i]
    
    Convergence_curve=numpy.zeros(Max_iter)

     # Loop counter
    print("GWO is optimizing  \""+objf.__name__+"\"")    
    
    timerStart=time.time() 
    # Main loop
    for l in range(0,Max_iter):
        for i in range(0,SearchAgents_no):
            
            # Return back the search agents that go beyond the boundaries of the search space
            for j in range(dim):
                Positions[i,j]=numpy.clip(Positions[i,j], lb[j], ub[j])

            # Calculate objective function for each search agent
            fitness,counter,df_dict=objf(Positions[i,:],counter,df_dict)
            
            # Update Alpha, Beta, and Delta
            if fitness<Alpha_score :
                Alpha_score=fitness; # Update alpha
                Alpha_pos=Positions[i,:].copy()
            
            
            if (fitness>Alpha_score and fitness<Beta_score ):
                Beta_score=fitness  # Update beta
                Beta_pos=Positions[i,:].copy()
            
            
            if (fitness>Alpha_score and fitness>Beta_score and fitness<Delta_score): 
                Delta_score=fitness # Update delta
                Delta_pos=Positions[i,:].copy()
            
        
        
        
        a=2-l*((2)/Max_iter); # a decreases linearly from 2 to 0
        
        # Update the Position of search agents including omegas
        for i in range(0,SearchAgents_no):
            for j in range (0,dim):     
    #--------------------------For Alpha-------------------------------
                r1=random.random() # r1 is a random number in [0,1]
                r2=random.random() # r2 is a random number in [0,1]
                
                A1=2*a*r1-a; # Equation (3.3)
                C1=2*r2; # Equation (3.4)
                
                D_alpha=abs(C1*Alpha_pos[j]-Positions[i,j]); # Equation (3.5)-part 1
          
                Bstep1 = sig_jBstepBGWO(A1 * D_alpha);#my change
          
                # X1=Alpha_pos[j]-A1*D_alpha; # Equation (3.6)-part 1
                X1 = sig_jBGWOupdate(Alpha_pos[j],Bstep1);
      
    #--------------------------For Beta-------------------------------                       
                r1=random.random()
                r2=random.random()
                
                A2=2*a*r1-a; # Equation (3.3)
                C2=2*r2; # Equation (3.4)
                
                D_beta=abs(C2*Beta_pos[j]-Positions[i,j]); # Equation (3.5)-part 2
                Bstep2 = sig_jBstepBGWO(A2 * D_beta);#my change 
                # X2=Beta_pos[j]-A2*D_beta; # Equation (3.6)-part 2       
                X2 = sig_jBGWOupdate(Beta_pos[j],Bstep2);
    
    #--------------------------For Delta-------------------------------            
                r1=random.random()
                r2=random.random() 
                
                A3=2*a*r1-a; # Equation (3.3)
                C3=2*r2; # Equation (3.4)
                
                D_delta=abs(C3*Delta_pos[j]-Positions[i,j]); # Equation (3.5)-part 3
                Bstep3 = sig_jBstepBGWO(A3 * D_delta);#my change
                X3=Delta_pos[j]-A3*D_delta; # Equation (3.5)-part 3   
                X3 = sig_jBGWOupdate(Delta_pos[j],Bstep3);

    #--------------------------Final updation in X-------------------------------
                # Positions[i,j]=(X1+X2+X3)/3  # Equation (3.7)
                r=random.random()
                if r < 1/3:
                  Positions[i,j] = X1
                elif r < 2/3  and  r >=1/3:
                  Positions[i,j] = X2
                else:
                  Positions[i,j] = X3
                
            
        
        
        Convergence_curve[l]=Alpha_score;

        if (l%1==0):
               print(['At iteration '+ str(l)+ ' the best fitness is '+ str(Alpha_score)]);
               print('alpha:', numpy.where(Alpha_pos>0.5)[0])
    
    timerEnd=time.time()
    print('Completed in', (timerEnd - timerStart))
    
    
    return Alpha_pos,counter,df_dict
    

Sigmoid/Binary iter First Time

Tanh iter First Time

In [35]:
count=0
dict_list={'counter':[],'accuracy':[]}
df_dict=pd.DataFrame(dict_list,columns=['counter','accuracy'])


fit,count,df_dict = sig_GWO(fitness_function, -10, 10, 22, 10, 1,count,df_dict)#lb,ub,dim,searchagent,iter
selected_features = np.where(fit>0.5)[0]
print("counter=>",count)
print(selected_features)
print(fit)
df_dict.to_csv('sig_accur_counter_first.csv')


GWO is optimizing  "fitness_function"
FEATURE LIST
['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'SMA20', 'SMA50', 'EMA20', 'EMA50', 'EMA200', 'UpperBB', 'LowerBB', 'RSI', 'MFI', 'ATR', 'Force_Index', 'MACD', 'MACD_SL', 'ADX', 'OBV', 'OBV_EMA']
(1232, 22)
0 Open
1 High
2 Low
3 Close
4 Adj Close
5 Volume
6 SMA20
7 SMA50
8 EMA20
9 EMA50
10 EMA200
11 UpperBB
12 LowerBB
13 RSI
14 MFI
15 ATR
16 Force_Index
17 MACD
18 MACD_SL
19 ADX
20 OBV
21 OBV_EMA
(961, 25, 22) (961,)
(246, 25, 22) (246,)
0.25588164877591546
0.25588164877591546
550 25 22
-------------------------------Training Start LSTM_1---------------------------
Epoch 1/10
61/61 [==============================] - 5s 23ms/step - loss: 0.1211 - val_loss: 6.9807e-04
Epoch 2/10
61/61 [==============================] - 1s 14ms/step - loss: 4.2220e-04 - val_loss: 7.5107e-04
Epoch 3/10
61/61 [==============================] - 1s 14ms/step - loss: 3.4571e-04 - val_loss: 6.3899e-04
Epoch 4/10
61/61 [==============================] - 

In [36]:
# count=0
# dict_list={'counter':[],'accuracy':[]}
# df_dict=pd.DataFrame(dict_list,columns=['counter','accuracy'])


# fit,count,df_dict = tanh_GWO(fitness_function, -10, 10, 23, 10, 1,count,df_dict)#lb,ub,dim,searchagent,iter
# selected_features = np.where(fit>0.5)[0]
# print("counter=>",count)
# print(selected_features)
# print(fit)
# df_dict.to_csv('tanh_accur_counter_first.csv')


Sigmoid/Binary iter Second Time

In [37]:
count=0
dict_list={'counter':[],'accuracy':[]}
df_dict=pd.DataFrame(dict_list,columns=['counter','accuracy'])


fit,count,df_dict = sig_GWO(fitness_function, -10, 10, 22, 10, 1,count,df_dict)#lb,ub,dim,searchagent,iter
selected_features = np.where(fit>0.5)[0]
print("counter=>",count)
print(selected_features)
print(fit)
df_dict.to_csv('sig_accur_counter_second.csv')


GWO is optimizing  "fitness_function"
FEATURE LIST
['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'SMA20', 'SMA50', 'EMA20', 'EMA50', 'EMA200', 'UpperBB', 'LowerBB', 'RSI', 'MFI', 'ATR', 'Force_Index', 'MACD', 'MACD_SL', 'ADX', 'OBV', 'OBV_EMA']
(1232, 22)
0 Open
1 High
2 Low
3 Close
4 Adj Close
5 Volume
6 SMA20
7 SMA50
8 EMA20
9 EMA50
10 EMA200
11 UpperBB
12 LowerBB
13 RSI
14 MFI
15 ATR
16 Force_Index
17 MACD
18 MACD_SL
19 ADX
20 OBV
21 OBV_EMA
(961, 25, 22) (961,)
(246, 25, 22) (246,)
0.25588164877591546
0.25588164877591546
550 25 22
-------------------------------Training Start LSTM_1---------------------------
Epoch 1/10
61/61 [==============================] - 6s 23ms/step - loss: 0.1830 - val_loss: 0.0025
Epoch 2/10
61/61 [==============================] - 1s 11ms/step - loss: 5.3914e-04 - val_loss: 6.7823e-04
Epoch 3/10
61/61 [==============================] - 1s 10ms/step - loss: 3.6717e-04 - val_loss: 6.9832e-04
Epoch 4/10
61/61 [==============================] - 1s 1

Tanh iter Second Time

In [38]:
# count=0
# dict_list={'counter':[],'accuracy':[]}
# df_dict=pd.DataFrame(dict_list,columns=['counter','accuracy'])


# fit,count,df_dict = tanh_GWO(fitness_function, -10, 10, 23, 10, 1,count,df_dict)#lb,ub,dim,searchagent,iter
# selected_features = np.where(fit>0.5)[0]
# print("counter=>",count)
# print(selected_features)
# print(fit)
# df_dict.to_csv('tanh_accur_counter_second.csv')


Sigmoid/Binary iter Third Time

In [39]:
count=0
dict_list={'counter':[],'accuracy':[]}
df_dict=pd.DataFrame(dict_list,columns=['counter','accuracy'])


fit,count,df_dict = sig_GWO(fitness_function, -10, 10, 22, 10, 1,count,df_dict)#lb,ub,dim,searchagent,iter
selected_features = np.where(fit>0.5)[0]
print("counter=>",count)
print(selected_features)
print(fit)
df_dict.to_csv('sig_accur_counter_third.csv')


GWO is optimizing  "fitness_function"
FEATURE LIST
['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'SMA20', 'SMA50', 'EMA20', 'EMA50', 'EMA200', 'UpperBB', 'LowerBB', 'RSI', 'MFI', 'ATR', 'Force_Index', 'MACD', 'MACD_SL', 'ADX', 'OBV', 'OBV_EMA']
(1232, 22)
0 Open
1 High
2 Low
3 Close
4 Adj Close
5 Volume
6 SMA20
7 SMA50
8 EMA20
9 EMA50
10 EMA200
11 UpperBB
12 LowerBB
13 RSI
14 MFI
15 ATR
16 Force_Index
17 MACD
18 MACD_SL
19 ADX
20 OBV
21 OBV_EMA
(961, 25, 22) (961,)
(246, 25, 22) (246,)
0.25588164877591546
0.25588164877591546
550 25 22
-------------------------------Training Start LSTM_1---------------------------
Epoch 1/10
61/61 [==============================] - 5s 23ms/step - loss: 0.1194 - val_loss: 8.9403e-04
Epoch 2/10
61/61 [==============================] - 1s 10ms/step - loss: 4.1262e-04 - val_loss: 8.4332e-04
Epoch 3/10
61/61 [==============================] - 1s 10ms/step - loss: 3.0910e-04 - val_loss: 4.9332e-04
Epoch 4/10
61/61 [==============================] - 

Tanh iter Third Time

In [40]:
# count=0
# dict_list={'counter':[],'accuracy':[]}
# df_dict=pd.DataFrame(dict_list,columns=['counter','accuracy'])


# fit,count,df_dict = tanh_GWO(fitness_function, -10, 10, 23, 10, 1,count,df_dict)#lb,ub,dim,searchagent,iter
# selected_features = np.where(fit>0.5)[0]
# print("counter=>",count)
# print(selected_features)
# print(fit)
# df_dict.to_csv('tanh_accur_counter_third.csv')


Sigmoid/Binary iter Fourth Time

In [41]:
count=0
dict_list={'counter':[],'accuracy':[]}
df_dict=pd.DataFrame(dict_list,columns=['counter','accuracy'])


fit,count,df_dict = sig_GWO(fitness_function, -10, 10, 22, 10, 1,count,df_dict)#lb,ub,dim,searchagent,iter
selected_features = np.where(fit>0.5)[0]
print("counter=>",count)
print(selected_features)
print(fit)
df_dict.to_csv('sig_accur_counter_fourth.csv')


GWO is optimizing  "fitness_function"
FEATURE LIST
['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'SMA20', 'SMA50', 'EMA20', 'EMA50', 'EMA200', 'UpperBB', 'LowerBB', 'RSI', 'MFI', 'ATR', 'Force_Index', 'MACD', 'MACD_SL', 'ADX', 'OBV', 'OBV_EMA']
(1232, 22)
0 Open
1 High
2 Low
3 Close
4 Adj Close
5 Volume
6 SMA20
7 SMA50
8 EMA20
9 EMA50
10 EMA200
11 UpperBB
12 LowerBB
13 RSI
14 MFI
15 ATR
16 Force_Index
17 MACD
18 MACD_SL
19 ADX
20 OBV
21 OBV_EMA
(961, 25, 22) (961,)
(246, 25, 22) (246,)
0.25588164877591546
0.25588164877591546
550 25 22
-------------------------------Training Start LSTM_1---------------------------
Epoch 1/10
61/61 [==============================] - 6s 22ms/step - loss: 0.0767 - val_loss: 0.0016
Epoch 2/10
61/61 [==============================] - 1s 10ms/step - loss: 4.2555e-04 - val_loss: 5.6261e-04
Epoch 3/10
61/61 [==============================] - 1s 10ms/step - loss: 3.4194e-04 - val_loss: 4.7669e-04
Epoch 4/10
61/61 [==============================] - 1s 1

Tanh iter Fourth Time

In [42]:
# count=0
# dict_list={'counter':[],'accuracy':[]}
# df_dict=pd.DataFrame(dict_list,columns=['counter','accuracy'])


# fit,count,df_dict = tanh_GWO(fitness_function, -10, 10, 23, 10, 1,count,df_dict)#lb,ub,dim,searchagent,iter
# selected_features = np.where(fit>0.5)[0]
# print("counter=>",count)
# print(selected_features)
# print(fit)
# df_dict.to_csv('tanh_accur_counter_fourth.csv')


Sigmoid/Binary iter Fifth Time

In [ ]:
count=0
dict_list={'counter':[],'accuracy':[]}
df_dict=pd.DataFrame(dict_list,columns=['counter','accuracy'])


fit,count,df_dict = sig_GWO(fitness_function, -10, 10, 22, 10, 1,count,df_dict)#lb,ub,dim,searchagent,iter
selected_features = np.where(fit>0.5)[0]
print("counter=>",count)
print(selected_features)
print(fit)
df_dict.to_csv('sig_accur_counter_fifth.csv')


GWO is optimizing  "fitness_function"
FEATURE LIST
['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'SMA20', 'SMA50', 'EMA20', 'EMA50', 'EMA200', 'UpperBB', 'LowerBB', 'RSI', 'MFI', 'ATR', 'Force_Index', 'MACD', 'MACD_SL', 'ADX', 'OBV', 'OBV_EMA']
(1232, 22)
0 Open
1 High
2 Low
3 Close
4 Adj Close
5 Volume
6 SMA20
7 SMA50
8 EMA20
9 EMA50
10 EMA200
11 UpperBB
12 LowerBB
13 RSI
14 MFI
15 ATR
16 Force_Index
17 MACD
18 MACD_SL
19 ADX
20 OBV
21 OBV_EMA
(961, 25, 22) (961,)
(246, 25, 22) (246,)
0.25588164877591546
0.25588164877591546
550 25 22
-------------------------------Training Start LSTM_1---------------------------
Epoch 1/10
61/61 [==============================] - 6s 24ms/step - loss: 0.0949 - val_loss: 9.9587e-04
Epoch 2/10
61/61 [==============================] - 1s 10ms/step - loss: 4.3633e-04 - val_loss: 6.4991e-04
Epoch 3/10
61/61 [==============================] - 1s 10ms/step - loss: 3.1778e-04 - val_loss: 5.6957e-04
Epoch 4/10
61/61 [==============================] - 

Tanh iter Fifth Time

In [ ]:
# count=0
# dict_list={'counter':[],'accuracy':[]}
# df_dict=pd.DataFrame(dict_list,columns=['counter','accuracy'])


# fit,count,df_dict = tanh_GWO(fitness_function, -10, 10, 23, 10, 1,count,df_dict)#lb,ub,dim,searchagent,iter
# selected_features = np.where(fit>0.5)[0]
# print("counter=>",count)
# print(selected_features)
# print(fit)
# df_dict.to_csv('tanh_accur_counter_fifth.csv')
